# Model Results

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

## Import Files

In [2]:
rfg = pd.read_csv("../../../Data_thesis/Full_Datasets/rfg_evalResults.csv")
rfc = pd.read_csv("../../../Data_thesis/Full_Datasets/rfc_evalResults.csv")
xgbr = pd.read_csv("../../../Data_thesis/Full_Datasets/xgbr_evalResults.csv")
xgbc = pd.read_csv("../../../Data_thesis/Full_Datasets/xgbc_evalResults.csv")

pred_rfg = pd.read_csv("../../../Data_thesis/Full_Datasets/rfg_Predictions.csv")
pred_rfg.sort_values(by=["Hour", "Sensor"], inplace=True)
pred_rfc = pd.read_csv("../../../Data_thesis/Full_Datasets/rfc_Predictions.csv")
pred_rfc.sort_values(by=["Hour", "Sensor"], inplace=True)
pred_xgbr = pd.read_csv("../../../Data_thesis/Full_Datasets/xgbr_Predictions.csv")
pred_xgbr.sort_values(by=["Hour", "Sensor"], inplace=True)
pred_xgbc = pd.read_csv("../../../Data_thesis/Full_Datasets/xgbc_Predictions.csv")
pred_xgbc.sort_values(by=["Hour", "Sensor"], inplace=True)
pred_lr = pd.read_csv("../../../Data_thesis/Full_Datasets/lr_Predictions.csv")
pred_lr.sort_values(by=["Hour", "Sensor"], inplace=True)
pred_dc = pd.read_csv("../../../Data_thesis/Full_Datasets/dc_Predictions.csv")
pred_dc.sort_values(by=["Hour", "Sensor"], inplace=True)

int_rfg = pd.read_csv("../../../Data_thesis/Full_Datasets/rfg_Interpolation.csv")
int_rfg.sort_values(by=["Hour", "Sensor"], inplace=True)
int_rfc = pd.read_csv("../../../Data_thesis/Full_Datasets/rfc_Interpolation.csv")
int_rfc.sort_values(by=["Hour", "Sensor"], inplace=True)
int_xgbr = pd.read_csv("../../../Data_thesis/Full_Datasets/xgbr_Interpolation.csv")
int_xgbr.sort_values(by=["Hour", "Sensor"], inplace=True)
int_xgbc = pd.read_csv("../../../Data_thesis/Full_Datasets/xgbc_Interpolation.csv")
int_xgbc.sort_values(by=["Hour", "Sensor"], inplace=True)
int_lr = pd.read_csv("../../../Data_thesis/Full_Datasets/lr_Interpolation.csv")
int_lr.sort_values(by=["Hour", "Sensor"], inplace=True)
int_dc = pd.read_csv("../../../Data_thesis/Full_Datasets/dc_Interpolation.csv")
int_dc.sort_values(by=["Hour", "Sensor"], inplace=True)

In [3]:
full_df = pd.read_csv("../../../Data_thesis/Full_Datasets/Full.csv")
full_df["Date"] = pd.to_datetime(full_df["Date"], format="%Y-%m-%d")
full_df = full_df[full_df["Date"] == pd.Timestamp(2019, 3, 25)].reset_index().drop(columns=["index"])
full_df["Date"] = full_df["Date"].astype(str)
full_df.rename(index=str, columns={"CrowdednessCount": "TrueCounts"}, inplace=True)
full_df.sort_values(by=["Hour", "Sensor"], inplace=True)
full_df = full_df[["Date", "Hour", "Sensor", "LonScaled", "LatScaled", "TrueCounts"]]

In [4]:
int_df = full_df[full_df["Sensor"] == "GAWW-01"]

In [5]:
pred_lr.head()

,Date,Hour,Sensor,SensorLongitude,SensorLatitude,CrowdednessCount
72,2019-03-25,100,GAWW-01,4.899847,52.374601,9.184583e+08
0,2019-03-25,100,GAWW-02,4.898903,52.373786,1.133812e+09
96,2019-03-25,100,GAWW-03,4.897334,52.372524,1.501142e+09
120,2019-03-25,100,GAWW-04,4.897908,52.373283,1.397605e+09
144,2019-03-25,100,GAWW-05,4.897717,52.372353,1.361921e+09


In [6]:
int_rfg.head()

,Date,Hour,Sensor,SensorLongitude,SensorLatitude,CrowdednessCount
0,2019-03-25,100,Custom,4.899847,52.374601,877.851429
1,2019-03-25,200,Custom,4.899847,52.374601,596.351429
2,2019-03-25,300,Custom,4.899847,52.374601,321.228571
3,2019-03-25,400,Custom,4.899847,52.374601,178.591429
4,2019-03-25,500,Custom,4.899847,52.374601,60.480000


In [14]:
int_df.head()

,Date,Hour,Sensor,LonScaled,LatScaled,TrueCounts
111,2019-03-25,100,GAWW-01,1.152297,1.430863,1324
118,2019-03-25,200,GAWW-01,1.152297,1.430863,815
104,2019-03-25,300,GAWW-01,1.152297,1.430863,286
90,2019-03-25,400,GAWW-01,1.152297,1.430863,197
97,2019-03-25,500,GAWW-01,1.152297,1.430863,82


In [13]:
int_xgbr.head()

,Date,Hour,Sensor,SensorLongitude,SensorLatitude,CrowdednessCount
0,2019-03-25,100,Custom,4.899847,52.374601,668.52740
1,2019-03-25,200,Custom,4.899847,52.374601,476.23782
2,2019-03-25,300,Custom,4.899847,52.374601,284.42410
3,2019-03-25,400,Custom,4.899847,52.374601,148.09674
4,2019-03-25,500,Custom,4.899847,52.374601,76.49449


In [6]:
def classCrowdednessCounts(df):
    low_split = df["TrueCounts"].quantile(.25)
    mid_split = df["TrueCounts"].quantile(.5)
    high_split = df["TrueCounts"].quantile(.75)

    clas_dict = df.to_dict("index")

    for k, v in clas_dict.items():
        if v["TrueCounts"] < low_split:
            v["TrueCounts"] = 1
        elif v["TrueCounts"] >= low_split and v["TrueCounts"] < mid_split:
            v["TrueCounts"] = 2
        elif v["TrueCounts"] >= mid_split and v["TrueCounts"] < high_split:
            v["TrueCounts"] = 3
        elif v["TrueCounts"] >= high_split:
            v["TrueCounts"] = 4
        else:
            print(k, " has class error as it fits in none")

    df = pd.DataFrame.from_dict(clas_dict, orient="index")

    return df

In [7]:
def getSensor(df):
    df["Sensor"] = "None"
    model_dict = df.to_dict("index")
    
    for k, v in model_dict.items():
        v["Sensor"] = full_df[full_df["LonScaled"] == v["Lon"]].reset_index()["Sensor"][0]
    
    df = pd.DataFrame.from_dict(model_dict, orient="index")
    
    return df

In [8]:
clas_df = classCrowdednessCounts(full_df)
int_clas = classCrowdednessCounts(int_df)
clas_df.sort_values(by=["Hour", "Sensor"], inplace=True)

In [12]:
clas_df.head()

,Date,Hour,Sensor,LonScaled,LatScaled,TrueCounts
111,2019-03-25,100,GAWW-01,1.152297,1.430863,4
112,2019-03-25,100,GAWW-02,0.279566,0.506281,4
113,2019-03-25,100,GAWW-03,-1.170458,-0.925578,3
114,2019-03-25,100,GAWW-04,-0.640018,-0.064080,2
115,2019-03-25,100,GAWW-05,-0.816523,-1.118800,4


In [13]:
pred_dc.head()

,Date,Hour,Sensor,SensorLongitude,SensorLatitude,CrowdednessCount
0,2019-03-25,100,Custom,4.899847,52.374601,3
1,2019-03-25,200,Custom,4.899847,52.374601,3
2,2019-03-25,300,Custom,4.899847,52.374601,3
3,2019-03-25,400,Custom,4.899847,52.374601,3
4,2019-03-25,500,Custom,4.899847,52.374601,3


## Results

In [17]:
r2 = round(r2_score(full_df["TrueCounts"], pred_lr["CrowdednessCount"]), 3)
rmse = round(np.sqrt(mean_squared_error(full_df["TrueCounts"], pred_lr["CrowdednessCount"])), 2)
print("R2: ", r2)
print("RMSE: ", rmse)

R2:  -2906013928247.174
RMSE:  1212282852.55


In [26]:
r2 = round(r2_score(int_df["TrueCounts"], int_lr["CrowdednessCount"]), 3)
rmse = round(np.sqrt(mean_squared_error(int_df["TrueCounts"], int_lr["CrowdednessCount"])), 2)
print("R2: ", r2)
print("RMSE: ", rmse)

R2:  -9728073758799.244
RMSE:  2491508563.96


In [32]:
acc = round(accuracy_score(int_clas["TrueCounts"], int_dc["CrowdednessCount"]),3)
prec = precision_score(int_clas["TrueCounts"], int_dc["CrowdednessCount"], average=None)
rec = recall_score(int_clas["TrueCounts"], int_dc["CrowdednessCount"], average=None)
f1 = f1_score(int_clas["TrueCounts"], int_dc["CrowdednessCount"], average=None)

print("Accuracy: ", acc)
print("Precision: ", prec)
print("Recall: ", rec)
print("F1 Score: ", f1)

Accuracy:  0.25
Precision:  [0.   0.   0.25 0.  ]
Recall:  [0. 0. 1. 0.]
F1 Score:  [0.  0.  0.4 0. ]


### RFG

In [20]:
rfg_dict = {}
sensors = full_df["Sensor"].unique()

for sensor in sensors:
    df = getSensor(rfg)
    df = df[df["Sensor"] == sensor].copy()
    
    r2 = round(r2_score(df["True"], df["Pred"]), 3)
    rmse = round(np.sqrt(mean_squared_error(df["True"], df["Pred"])), 2)

    rfg_dict[sensor] = {"R2": r2, "RMSE": rmse}
    
rfg_dict

{'GAWW-01': {'R2': 0.814, 'RMSE': 486.26},
 'GAWW-02': {'R2': 0.839, 'RMSE': 557.4},
 'GAWW-03': {'R2': 0.908, 'RMSE': 235.44},
 'GAWW-04': {'R2': 0.852, 'RMSE': 124.56},
 'GAWW-05': {'R2': 0.909, 'RMSE': 294.28},
 'GAWW-06': {'R2': 0.759, 'RMSE': 501.56},
 'GAWW-07': {'R2': 0.927, 'RMSE': 193.06}}

In [18]:
r2 = round(r2_score(full_df["TrueCounts"], pred_rfg["CrowdednessCount"]), 3)
rmse = round(np.sqrt(mean_squared_error(full_df["TrueCounts"], pred_rfg["CrowdednessCount"])), 2)
print("R2: ", r2)
print("RMSE: ", rmse)

R2:  0.507
RMSE:  499.28


In [9]:
r2 = round(r2_score(int_df["TrueCounts"], int_rfg["CrowdednessCount"]), 3)
rmse = round(np.sqrt(mean_squared_error(int_df["TrueCounts"], int_rfg["CrowdednessCount"])), 2)
print("R2: ", r2)
print("RMSE: ", rmse)

R2:  0.588
RMSE:  512.79


### XGBR

In [11]:
xgbr_dict = {}
sensors = full_df["Sensor"].unique()

for sensor in sensors:
    df = getSensor(xgbr)
    df = df[df["Sensor"] == sensor].copy()
    
    r2 = round(r2_score(df["True"], df["Pred"]), 3)
    rmse = round(np.sqrt(mean_squared_error(df["True"], df["Pred"])), 2)

    xgbr_dict[sensor] = {"R2": r2, "RMSE": rmse}
    
xgbr_dict

{'GAWW-01': {'R2': 0.839, 'RMSE': 443.98},
 'GAWW-02': {'R2': 0.804, 'RMSE': 594.52},
 'GAWW-03': {'R2': 0.86, 'RMSE': 297.15},
 'GAWW-04': {'R2': 0.806, 'RMSE': 134.28},
 'GAWW-05': {'R2': 0.877, 'RMSE': 335.58},
 'GAWW-06': {'R2': 0.688, 'RMSE': 552.19},
 'GAWW-07': {'R2': 0.912, 'RMSE': 207.55}}

In [12]:
r2 = round(r2_score(full_df["TrueCounts"], pred_xgbr["CrowdednessCount"]), 3)
rmse = round(np.sqrt(mean_squared_error(full_df["TrueCounts"], pred_xgbr["CrowdednessCount"])), 2)
print("R2: ", r2)
print("RMSE: ", rmse)

R2:  0.416
RMSE:  543.57


In [12]:
r2 = round(r2_score(int_df["TrueCounts"], int_xgbr["CrowdednessCount"]), 3)
rmse = round(np.sqrt(mean_squared_error(int_df["TrueCounts"], int_xgbr["CrowdednessCount"])), 2)
print("R2: ", r2)
print("RMSE: ", rmse)

R2:  0.061
RMSE:  774.14


### RFC

In [21]:
rfc_dict = {}
sensors = full_df["Sensor"].unique()

for sensor in sensors:
    df = getSensor(rfc)
    df = df[df["Sensor"] == sensor].copy()

    acc = round(accuracy_score(df["True"], df["Pred"]),3)
    prec = precision_score(df["True"], df["Pred"], average=None)
    rec = recall_score(df["True"], df["Pred"], average=None)
    f1 = f1_score(df["True"], df["Pred"], average=None)
    
    rfc_dict[sensor] = {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1 Score": f1}
    
rfc_dict

{'GAWW-01': {'Accuracy': 0.855,
  'Precision': array([0.91776316, 0.84813084, 0.81898455, 0.85483871]),
  'Recall': array([0.6957606 , 0.88536585, 0.82444444, 0.94383562]),
  'F1 Score': array([0.79148936, 0.86634845, 0.82170543, 0.89713542])},
 'GAWW-02': {'Accuracy': 0.871,
  'Precision': array([0.7877551 , 0.87603306, 0.81741573, 0.90758755]),
  'Recall': array([0.80416667, 0.77750611, 0.76781003, 0.96784232]),
  'F1 Score': array([0.79587629, 0.8238342 , 0.79183673, 0.93674699])},
 'GAWW-03': {'Accuracy': 0.891,
  'Precision': array([0.9707401 , 0.82608696, 0.70026525, 0.86036036]),
  'Recall': array([0.9707401 , 0.72243346, 0.83018868, 0.77016129]),
  'F1 Score': array([0.9707401 , 0.77079108, 0.75971223, 0.81276596])},
 'GAWW-04': {'Accuracy': 0.869,
  'Precision': array([0.94727592, 0.87839102, 0.71181556, 1.        ]),
  'Recall': array([0.89090909, 0.89173789, 0.76234568, 0.66666667]),
  'F1 Score': array([0.91822828, 0.88501414, 0.73621461, 0.8       ])},
 'GAWW-05': {'Accura

In [20]:
acc = round(accuracy_score(clas_df["TrueCounts"], pred_rfc["CrowdednessCount"]),3)
prec = precision_score(clas_df["TrueCounts"], pred_rfc["CrowdednessCount"], average=None)
rec = recall_score(clas_df["TrueCounts"], pred_rfc["CrowdednessCount"], average=None)
f1 = f1_score(clas_df["TrueCounts"], pred_rfc["CrowdednessCount"], average=None)

print("Accuracy: ", acc)
print("Precision: ", prec)
print("Recall: ", rec)
print("F1 Score: ", f1)

Accuracy:  0.333
Precision:  [0.51020408 0.19277108 0.41666667 0.        ]
Recall:  [0.5952381  0.38095238 0.35714286 0.        ]
F1 Score:  [0.54945055 0.256      0.38461538 0.        ]


D:\Programs\Anaconda\envs\Analytics\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\envs\Analytics\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [35]:
acc = round(accuracy_score(int_clas["TrueCounts"], int_rfc["CrowdednessCount"]),3)
prec = precision_score(int_clas["TrueCounts"], int_rfc["CrowdednessCount"], average=None)
rec = recall_score(int_clas["TrueCounts"], int_rfc["CrowdednessCount"], average=None)
f1 = f1_score(int_clas["TrueCounts"], int_rfc["CrowdednessCount"], average=None)

print("Accuracy: ", acc)
print("Precision: ", prec)
print("Recall: ", rec)
print("F1 Score: ", f1)

Accuracy:  0.333
Precision:  [0.5        0.21428571 0.5        0.        ]
Recall:  [0.5        0.5        0.33333333 0.        ]
F1 Score:  [0.5 0.3 0.4 0. ]


D:\Programs\Anaconda\envs\Analytics\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\envs\Analytics\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### XGBC

In [13]:
xgbc_dict = {}
sensors = full_df["Sensor"].unique()

for sensor in sensors:
    df = getSensor(xgbc)
    df = df[df["Sensor"] == sensor].copy()

    acc = round(accuracy_score(df["True"], df["Pred"]),3)
    prec = precision_score(df["True"], df["Pred"], average=None)
    rec = recall_score(df["True"], df["Pred"], average=None)
    f1 = f1_score(df["True"], df["Pred"], average=None)
    
    xgbc_dict[sensor] = {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1 Score": f1}
    
xgbc_dict

D:\Programs\Anaconda\envs\Analytics\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\envs\Analytics\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


{'GAWW-01': {'Accuracy': 0.832,
  'Precision': array([0.85765125, 0.83160622, 0.7925    , 0.84391534]),
  'Recall': array([0.70467836, 0.83376623, 0.76019185, 0.93961708]),
  'F1 Score': array([0.77367576, 0.83268482, 0.77600979, 0.88919861])},
 'GAWW-02': {'Accuracy': 0.804,
  'Precision': array([0.72030651, 0.75882353, 0.71283784, 0.87378641]),
  'Recall': array([0.78008299, 0.70108696, 0.5960452 , 0.94076655]),
  'F1 Score': array([0.74900398, 0.72881356, 0.64923077, 0.90604027])},
 'GAWW-03': {'Accuracy': 0.807,
  'Precision': array([0.88638413, 0.74096386, 0.59672131, 0.75308642]),
  'Recall': array([0.9889336 , 0.48046875, 0.57413249, 0.71484375]),
  'F1 Score': array([0.93485497, 0.58293839, 0.585209  , 0.73346693])},
 'GAWW-04': {'Accuracy': 0.807,
  'Precision': array([0.92664093, 0.77817531, 0.65240642, 0.        ]),
  'Recall': array([0.83623693, 0.90719499, 0.42807018, 0.        ]),
  'F1 Score': array([0.87912088, 0.83774675, 0.51694915, 0.        ])},
 'GAWW-05': {'Accura

In [14]:
acc = round(accuracy_score(clas_df["TrueCounts"], pred_xgbc["CrowdednessCount"]),3)
prec = precision_score(clas_df["TrueCounts"], pred_xgbc["CrowdednessCount"], average=None)
rec = recall_score(clas_df["TrueCounts"], pred_xgbc["CrowdednessCount"], average=None)
f1 = f1_score(clas_df["TrueCounts"], pred_xgbc["CrowdednessCount"], average=None)

print("Accuracy: ", acc)
print("Precision: ", prec)
print("Recall: ", rec)
print("F1 Score: ", f1)

Accuracy:  0.25
Precision:  [0.41791045 0.12121212 0.         1.        ]
Recall:  [0.66666667 0.28571429 0.         0.04761905]
F1 Score:  [0.51376147 0.17021277 0.         0.09090909]


In [11]:
acc = round(accuracy_score(int_clas["TrueCounts"], int_xgbc["CrowdednessCount"]),3)
prec = precision_score(int_clas["TrueCounts"], int_xgbc["CrowdednessCount"], average=None)
rec = recall_score(int_clas["TrueCounts"], int_xgbc["CrowdednessCount"], average=None)
f1 = f1_score(int_clas["TrueCounts"], int_xgbc["CrowdednessCount"], average=None)

print("Accuracy: ", acc)
print("Precision: ", prec)
print("Recall: ", rec)
print("F1 Score: ", f1)

Accuracy:  0.333
Precision:  [0.5    0.1875 0.     1.    ]
Recall:  [0.5        0.5        0.         0.33333333]
F1 Score:  [0.5        0.27272727 0.         0.5       ]


D:\Programs\Anaconda\envs\Analytics\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Programs\Anaconda\envs\Analytics\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
